## Study Visualizations

### Import Packages
The Python packages required by this notebook are imported. The package and Python interpreter versions are printed to ensure analysis reproducibility.

In [1]:
import os
import sys
import pandas
import plotly
import plotly.express as px
import plotly.graph_objects as go
import IPython
from collections import OrderedDict
print(f"Python version: {sys.version}.")
print(f"IPython package version: {IPython.__version__}.")
print(f"pandas package version: {pandas.__version__}.")
print(f"plotly package version: {plotly.__version__}.")

Python version: 3.8.8 (default, Apr 13 2021, 19:58:26) 
[GCC 7.3.0].
IPython package version: 7.22.0.
pandas package version: 1.2.4.
plotly package version: 4.14.3.


### Define and Load Data Sources
The data files from the case specific `postProcessing` folder are loaded into pandas dataframes.

In [2]:
# Define data file information
data_file_info = OrderedDict()
data_file_info = {'inlet1_massflow':["Time [s]", "inlet1 Mass Flow [kg/s]"],
                  'inlet2_massflow':["Time [s]", "inlet2 Mass Flow [kg/s]"],
                  'outlet_massflow':["Time [s]", "outlet Mass Flow [kg/s]"],
                  'outlet_temperature':["Time [s]", "outlet Average Temperature [K]"],
                  'residuals':["Time [s]", "Ux", "Uy","Uz","p_rgh","e", "k", "epsilon"],
                  'pressure_drop':["Time [s]", "inlet1-outlet Pressure Drop [Pa]"]}

# Create simulation data dictionary
simulation_data = OrderedDict()
# Create case list
case_list = []
# Traverse simulation repository, po;ulate case list and load case data
for root, dirs, files in os.walk("../", topdown=True):
    for name in files:
        for data_source, source_headers in data_file_info.items():
            file_path = os.path.join(root, name)
            file_match = '/postProcessing/' + str(data_source) + '/'
            if file_match in file_path:
                case_name = file_path.split('/')[1]
                if case_name not in case_list:
                    case_list.append(case_name)
                case_id = case_name + ':' + data_source
                data_source_data = pandas.read_csv(file_path, delim_whitespace=True, comment ='#',
                                   header = None, names = source_headers, float_precision='round_trip')
                simulation_data[case_id] = data_source_data

#print(simulation_data)
#print(case_list)
#test_df = simulation_data['symmetric-velocity-temperature:inlet1_massflow']
# test_df = simulation_data[case_list[0]+':inlet1_massflow']
#test_df = simulation_data[str(case_list[0]) + ':' + str(data_file_info['inlet1_massflow'])]
#display(test_df)

### Create Plots
Create interactive plots for the simulation data.

In [3]:
# Set figure dimensions for inline display
figure_width = 1000
figure_height = 900

In [4]:
# Plot outlet temperature
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = simulation_data['symmetric-velocity-temperature:outlet_temperature']['Time [s]'],
        y = simulation_data['symmetric-velocity-temperature:outlet_temperature']['outlet Average Temperature [K]'],
        mode = 'lines'
    )
)

fig.update_layout(
    title_text='Figure 1: Average Outlet Temperature',
    title_x=0.5,
    xaxis_title='Time [s]',
    yaxis_title='Temperature [K]',
)

fig.write_html('01-outlet-temperature.html')
IPython.display.IFrame('01-outlet-temperature.html', width=figure_width, height=figure_height)

In [5]:
# Plot residuals
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = simulation_data['symmetric-velocity-temperature:residuals']['Time [s]'],
        y = simulation_data['symmetric-velocity-temperature:residuals']['Ux'],
        mode = 'lines',
        name = 'Ux'
    )
)

fig.add_trace(
    go.Scatter(
        x = simulation_data['symmetric-velocity-temperature:residuals']['Time [s]'],
        y = simulation_data['symmetric-velocity-temperature:residuals']['Uy'],
        mode = 'lines',
        name = 'Uy'
    )
)

fig.add_trace(
    go.Scatter(
        x = simulation_data['symmetric-velocity-temperature:residuals']['Time [s]'],
        y = simulation_data['symmetric-velocity-temperature:residuals']['Uz'],
        mode = 'lines',
        name = 'Uz'
    )
)

fig.add_trace(
    go.Scatter(
        x = simulation_data['symmetric-velocity-temperature:residuals']['Time [s]'],
        y = simulation_data['symmetric-velocity-temperature:residuals']['p_rgh'],
        mode = 'lines',
        name = 'p_rgh'
    )
)

fig.add_trace(
    go.Scatter(
        x = simulation_data['symmetric-velocity-temperature:residuals']['Time [s]'],
        y = simulation_data['symmetric-velocity-temperature:residuals']['e'],
        mode = 'lines',
        name = 'e'
    )
)

fig.add_trace(
    go.Scatter(
        x = simulation_data['symmetric-velocity-temperature:residuals']['Time [s]'],
        y = simulation_data['symmetric-velocity-temperature:residuals']['k'],
        mode = 'lines',
        name = 'k'
    )
)

fig.add_trace(
    go.Scatter(
        x = simulation_data['symmetric-velocity-temperature:residuals']['Time [s]'],
        y = simulation_data['symmetric-velocity-temperature:residuals']['epsilon'],
        mode = 'lines',
        name = 'epsilon'
    )
)


fig.update_layout(
    title_text='Figure 2: Residuals',
    title_x=0.5,
    xaxis_title='Time [s]',
    yaxis_title='Residual Magnitude',
    yaxis_type="log"
)

fig.update_yaxes(
    showexponent='all',
    exponentformat='e',
    minexponent=1
)

fig.write_html('02-residuals.html')
IPython.display.IFrame('02-residuals.html', width=figure_width, height=figure_height)

In [6]:
# Plot mass flow
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = simulation_data['symmetric-velocity-temperature:inlet1_massflow']['Time [s]'],
        y = simulation_data['symmetric-velocity-temperature:inlet1_massflow']['inlet1 Mass Flow [kg/s]'],
        mode = 'lines',
        name = 'inlet1'
    )
)

fig.add_trace(
    go.Scatter(
        x = simulation_data['symmetric-velocity-temperature:inlet2_massflow']['Time [s]'],
        y = simulation_data['symmetric-velocity-temperature:inlet2_massflow']['inlet2 Mass Flow [kg/s]'],
        mode = 'lines',
        name = 'inlet2'
    )
)

fig.add_trace(
    go.Scatter(
        x = simulation_data['symmetric-velocity-temperature:outlet_massflow']['Time [s]'],
        y = simulation_data['symmetric-velocity-temperature:outlet_massflow']['outlet Mass Flow [kg/s]'],
        mode = 'lines',
        name = 'outlet'
    )
)


fig.update_layout(
    title_text='Figure 3: Inlet and Outlet Mass Flow Rates',
    title_x=0.5,
    xaxis_title='Time [s]',
    yaxis_title='Mass Flow [kg/s]',
)

fig.write_html('03-mass-flow-rates.html')
IPython.display.IFrame('03-mass-flow-rates.html', width=figure_width, height=figure_height)

In [7]:
# Pressure Drop
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = simulation_data['symmetric-velocity-temperature:pressure_drop']['Time [s]'],
        y = simulation_data['symmetric-velocity-temperature:pressure_drop']['inlet1-outlet Pressure Drop [Pa]'],
        mode = 'lines'
    )
)

fig.update_layout(
    title_text='Figure 4: Inlet - Outlet Pressure Drop',
    title_x=0.5,
    xaxis_title='Time [s]',
    yaxis_title='Pressure Difference [Pa]',
)

fig.update_yaxes(
    showexponent='all',
    exponentformat='e',
    minexponent=5
)

fig.write_html('04-pressure-difference.html')
IPython.display.IFrame('04-pressure-difference.html', width=figure_width, height=figure_height)